In [1]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-x6wpo_7y
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-x6wpo_7y
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.2 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=30e1122192af2714ac8603fc7dce1d81ad6dcad3f9ace025119f74c77f4fb2ea
  Stored in directory: /tmp/pip-ephem-wheel-cache-0xayjltx/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [2]:
pip install -U albumentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.9/227.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.7/632.7 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.7/291.7 kB 20.6 MB/s eta 0:00:00
  Attempting uninstall: albucore
    Found existing installation: albucore 0.0.17
    Uninstalling albucore-0.0.17:
      Successfully uninstalled albucore-0.0.17
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.17
    Uninstalling albumentations-1.4.17:
      Successfully uninstalled albumentations-1.4.17
Note: you may need to restart the kernel to use updated packages.


# Import

In [3]:
import os

import gc

import sys

from PIL import Image

import cv2

import math, random

import numpy as np

import pandas as pd

from glob import glob

from tqdm import tqdm

import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold



from collections import OrderedDict



import torch

import torch.nn.functional as F

from torch import nn

from torch.utils.data import DataLoader, Dataset

from torch.optim import AdamW



import timm

from transformers import get_cosine_schedule_with_warmup



import albumentations as A

from sklearn.preprocessing import LabelEncoder

import clip



from torch.cuda.amp import autocast, GradScaler

from torch.utils.data import DataLoader

import wandb

from tqdm import tqdm

In [4]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

DEBUG = False

N_WORKERS = os.cpu_count() 

USE_AMP = False # can change True if using T4 or newer than Ampere

SEED = 42

CLIP_MODEL_NAME = "RN101"

IMG_SIZE = [224, 224]

AUG_PROB = 0.2

BATCH_SIZE = 64

EPOCHS = 50

if DEBUG:

    EPOCHS = 3

GRAD_ACC = 1

N_FOLDS = 5

MAX_GRAD_NORM = None

EARLY_STOPPING_EPOCH = 3

OUTPUT_DIR = f'clip_landmark_results'

In [5]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [6]:
train_df = pd.read_csv("/kaggle/input/google-landmarks-v2-index-subset-metadata/cleaned_landmark_index_subset_150k_with_locations_v4.csv")

if DEBUG:

    # Reduce to 1000 samples for quick debugging

    train_df = train_df.sample(1000, random_state=42).reset_index(drop=True)

train_df.head()

,id,landmark_id,category,supercategory,hierarchical_label,natural_or_human_made,cleaned_supercategory,location,country,image_path,landmark_name
0,87a5dfadcaeba144,93036,http://commons.wikimedia.org/wiki/Category:Nav...,naval base,harbor,human-made,naval base,"Guantánamo Province , Cuba",Cuba,/kaggle/input/google-landmarks-v2-index-set-p3...,Naval Station Guantanamo Bay
1,53824bfa2b3e569c,93036,http://commons.wikimedia.org/wiki/Category:Nav...,naval base,harbor,human-made,naval base,"Guantánamo Province , Cuba",Cuba,/kaggle/input/google-landmarks-v2-index-set-p2...,Naval Station Guantanamo Bay
2,080a99cff9666667,93036,http://commons.wikimedia.org/wiki/Category:Nav...,naval base,harbor,human-made,naval base,"Guantánamo Province , Cuba",Cuba,/kaggle/input/google-landmarks-v2-index-set-p1...,Naval Station Guantanamo Bay
3,b86710311289fa90,93036,http://commons.wikimedia.org/wiki/Category:Nav...,naval base,harbor,human-made,naval base,"Guantánamo Province , Cuba",Cuba,/kaggle/input/google-landmarks-v2-index-set-p4...,Naval Station Guantanamo Bay
4,5554c986662c2587,93036,http://commons.wikimedia.org/wiki/Category:Nav...,naval base,harbor,human-made,naval base,"Guantánamo Province , Cuba",Cuba,/kaggle/input/google-landmarks-v2-index-set-p2...,Naval Station Guantanamo Bay


In [7]:
N_CLASSES = train_df.landmark_name.nunique()

class_names = train_df.landmark_name.unique()

In [8]:
# Load CLIP model

embedding_device = device

model, _ = clip.load(CLIP_MODEL_NAME, device=embedding_device)



# Define class names and tokenize

text_prompts = [f"A photo of {name} or its part" for name in class_names]

text_tokens = clip.tokenize(text_prompts).to(embedding_device)



# Compute text embeddings

with torch.no_grad():

    text_embeddings = model.encode_text(text_tokens)

text_embeddings = text_embeddings / text_embeddings.norm(dim=-1, keepdim=True)  # Normalize for cosine similarity

text_embeddings = text_embeddings.float()
torch.save(text_embeddings,'text_embeddings.pt')

100%|███████████████████████████████████████| 278M/278M [00:03<00:00, 75.7MiB/s]


In [9]:
"""text_embeddings = torch.load("/kaggle/input/text_embeddings/pytorch/default/1/text_embeddings.pt", weights_only=True)"""

'text_embeddings = torch.load("/kaggle/input/text_embeddings/pytorch/default/1/text_embeddings.pt", weights_only=True)'

In [10]:
class_name_to_idx = {name: idx for idx, name in enumerate(class_names)}

idx_to_class_name = {idx: class_name for class_name, idx in class_name_to_idx.items()}

In [11]:
transforms_train = A.Compose([

    A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=AUG_PROB),

    A.OneOf([

        A.MotionBlur(blur_limit=5),

        A.MedianBlur(blur_limit=5),

        A.GaussianBlur(blur_limit=5),

        A.GaussNoise(var_limit=(5.0, 30.0)),

    ], p=AUG_PROB),



    A.OneOf([

        A.OpticalDistortion(distort_limit=1.0),

        A.GridDistortion(num_steps=5, distort_limit=1.),

        A.ElasticTransform(alpha=3),

    ], p=AUG_PROB),



    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=AUG_PROB),

    A.Resize(IMG_SIZE[0], IMG_SIZE[1]),

    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, min_holes=1, min_height=8, min_width=8, p=AUG_PROB),    

    A.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], 

                         std=[0.26862954, 0.26130258, 0.27577711])

])



transforms_val = A.Compose([

    A.Resize(IMG_SIZE[0], IMG_SIZE[1]),

    A.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], 

                         std=[0.26862954, 0.26130258, 0.27577711])

])

/opt/conda/lib/python3.10/site-packages/pydantic/main.py:212: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [12]:
class GoogleLandmarksDataset(Dataset):

    def __init__(self, df, class_name_to_idx, transform=None):

        self.df = df

        self.class_name_to_idx = class_name_to_idx  # Maps class name to index

        self.transform = transform

    

    def __len__(self):

        return len(self.df)



    def __getitem__(self, idx):

        t = self.df.iloc[idx]

        class_name = t['landmark_name']

        class_name_idx = self.class_name_to_idx[class_name]

        

        p = f'{t["image_path"]}'

        img = Image.open(p)

        img = np.array(img)

            

        # Apply transformations if provided

        if self.transform:

            img = self.transform(image=np.array(img))['image']

        

        # Convert image to tensor if needed

        if not isinstance(img, torch.Tensor):

            img = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32)

        

        return img, class_name_idx


In [13]:
class CrossModalDistillationModel(nn.Module):
    def __init__(self, teacher_model, student_model, text_embeddings, projection_dim=512):
        super().__init__()
        self.teacher = teacher_model
        self.student = student_model
        self.text_embeddings = text_embeddings
        
        # Freeze teacher
        for param in self.teacher.parameters():
            param.requires_grad = False
        
        # Get student features dimension
        in_features = student_model.get_classifier().in_features
        num_classes = student_model.get_classifier().out_features
        
        # Improved projection head with residual connections
        self.projection_head = nn.Sequential(
            nn.Linear(in_features, 1024),
            nn.LayerNorm(1024),  # Layer norm instead of batch norm
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 1024),
            nn.LayerNorm(1024),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(1024, projection_dim)
        )
        
        # Improved classifier with bottleneck
        self.classifier = nn.Sequential(
            nn.Linear(in_features, 1024),
            nn.LayerNorm(1024),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(1024, num_classes)
        )
        
        # Temperature parameter with better initialization
        self.temperature = nn.Parameter(torch.log(torch.ones([]) / 0.07))
        
        # Initialize weights properly
        self._initialize_weights()
    
    def _initialize_weights(self):
        """
        Proper weight initialization for better training
        """
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.LayerNorm):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, images, return_features=True):
        batch_size = images.size(0)
        
        # Teacher forward pass with gradient checkpointing for memory efficiency
        with torch.no_grad(), torch.cuda.amp.autocast():
            teacher_logits, teacher_features = self.teacher(images)
            
        # Student forward pass
        student_features = self.student.forward_features(images)
        
        # Global average pooling with better numerical stability
        student_features = F.adaptive_avg_pool2d(student_features, (1, 1))
        student_features = student_features.view(batch_size, -1)
        
        # Project student features
        student_projected = self.projection_head(student_features)
        student_projected = F.normalize(student_projected, dim=-1)
        
        # Compute classification logits
        student_logits = self.classifier(student_features)
        
        if return_features:
            return {
                'teacher_logits': teacher_logits,
                'student_logits': student_logits,
                'teacher_features': teacher_features,
                'student_projected': student_projected,
                'student_features': student_features,
                'temperature': self.temperature.exp()  # Useful for monitoring
            }
        return student_logits
    
    def get_last_selfattention(self, x):
        """
        Get attention maps for visualization
        """
        return self.student.get_last_selfattention(x)

In [14]:
class GoogleLandmarksModel(nn.Module):
    def __init__(self, text_embeddings, model_name=CLIP_MODEL_NAME, device=device):
        super(GoogleLandmarksModel, self).__init__()
        # Load the CLIP model
        self.model, _ = clip.load(model_name, device=device)
        
        # Store the precomputed text embeddings
        self.text_embeddings = text_embeddings
        self.text_embeddings.requires_grad = False
        
        # Freeze the text encoder to save memory and computation
        for param in self.model.transformer.parameters():
            param.requires_grad = False

    def forward(self, images):
        # Compute image embeddings
        image_embeddings = self.model.encode_image(images)
        
        # Normalize image embeddings without in-place operation
        image_embeddings = image_embeddings / image_embeddings.norm(dim=-1, keepdim=True)
        
        # Check and match data types for mixed precision
        if image_embeddings.dtype != self.text_embeddings.dtype:
            text_embeddings = self.text_embeddings.to(image_embeddings.dtype)
        else:
            text_embeddings = self.text_embeddings
        
        # Compute cosine similarity with text embeddings
        similarity = image_embeddings @ self.text_embeddings.T
        return similarity, image_embeddings  # Similarity scores for each class

In [15]:
class Config:

    def __init__(self):

        self.alpha = 0.4

        self.beta = 0.2

        self.gamma = 0.4

        self.temperature = 2.0

        self.device = device

        self.batch_size = BATCH_SIZE

        self.n_workers = N_WORKERS

        self.max_grad_norm = MAX_GRAD_NORM

        self.log_interval = 100

        self.use_amp = False

        self.use_wandb = False  # Set to True if using wandb

        self.warmup_steps = 100

        self.gradient_accumulation_steps = 1

In [16]:
class NKDLoss:
    def __init__(self, alpha=1.0, lambda_temp=1.0):
        """
        Initializes NKD Loss.
        Args:
            alpha: Hyperparameter for scaling Ldistributed.
            lambda_temp: Temperature for knowledge distillation.
        """
        self.alpha = alpha
        self.lambda_temp = lambda_temp

    def __call__(self, student_logits, teacher_logits, targets):
        """
        Compute NKD Loss.
        Args:
            student_logits: Logits from the student model. Shape: (batch_size, num_classes)
            teacher_logits: Logits from the teacher model. Shape: (batch_size, num_classes)
            targets: Ground-truth labels. Shape: (batch_size,)
        Returns:
            Total NKD loss (scalar).
        """
        # Apply temperature scaling
        student_probs = F.softmax(student_logits / self.lambda_temp, dim=-1)
        teacher_probs = F.softmax(teacher_logits / self.lambda_temp, dim=-1)
        
        # Ground-truth one-hot labels
        target_probs = F.one_hot(targets, num_classes=student_logits.size(1)).float()
        
        # Extract probabilities for the target class
        student_target_probs = student_probs.gather(1, targets.view(-1, 1)).squeeze(1)
        teacher_target_probs = teacher_probs.gather(1, targets.view(-1, 1)).squeeze(1)
        
        # Compute Lori: Original loss
        lori = -torch.log(student_target_probs + 1e-8).mean()
        
        # Compute Lsoft: Soft loss
        lsoft = -(teacher_target_probs * torch.log(student_target_probs + 1e-8)).mean()
        
        # Compute Ldistributed: Distributed loss
        # Exclude target class for teacher and student
        non_target_teacher_probs = teacher_probs * (1 - target_probs)
        non_target_student_probs = student_probs * (1 - target_probs)
        
        # Normalize non-target probabilities
        normalized_teacher_probs = non_target_teacher_probs / (1 - teacher_target_probs.unsqueeze(1) + 1e-8)
        normalized_student_probs = non_target_student_probs / (1 - student_target_probs.unsqueeze(1) + 1e-8)
        
        # Distributed loss
        ldistributed = -(normalized_teacher_probs * torch.log(normalized_student_probs + 1e-8)).sum(dim=1).mean()
        ldistributed *= self.alpha * (self.lambda_temp ** 2)
        
        # Total loss
        lnkd = lori + lsoft + ldistributed
        return lnkd

In [17]:
class EnhancedDistillationLoss(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.alpha = config.alpha  # KD loss weight
        self.beta = config.beta    # Feature alignment weight
        self.gamma = config.gamma  # CE loss weight
        self.temperature = config.temperature
        self.contrastive_weight = 0.1
        
    def contrastive_loss(self, features, labels):
        """
        Improved contrastive loss with better numerical stability
        """
        # Normalize features for better stability
        features = F.normalize(features, dim=1)
        
        # Compute similarity matrix
        similarity = torch.matmul(features, features.t())
        
        # Create label mask for positive pairs
        labels = labels.view(-1, 1)
        mask = (labels == labels.T).float()
        
        # Remove diagonal from mask (self-pairs)
        mask = mask.fill_diagonal_(0)
        
        # Temperature-scaled similarities with numerical stability
        similarity = similarity / self.temperature
        
        # Stable version of exp(similarity)
        sim_exp = torch.exp(similarity - similarity.max(dim=1, keepdim=True)[0])
        
        # Compute positive and negative pairs
        pos_sim = sim_exp * mask
        neg_sim = sim_exp * (1 - mask)
        
        # Add small epsilon to prevent log(0)
        eps = 1e-8
        loss = -torch.log(
            (pos_sim.sum(1) + eps) / (pos_sim.sum(1) + neg_sim.sum(1) + eps)
        ).mean()
        
        return loss

    def feature_alignment_loss(self, student_features, teacher_features):
        """
        Improved feature alignment using cosine similarity
        """
        student_features = F.normalize(student_features, dim=1)
        teacher_features = F.normalize(teacher_features, dim=1)
        return 1 - (student_features * teacher_features).sum(dim=1).mean()

    def forward(self, outputs, labels):
        # Convert all inputs to float and ensure contiguous memory
        teacher_logits = outputs['teacher_logits'].float().contiguous()
        student_logits = outputs['student_logits'].float().contiguous()
        teacher_features = outputs['teacher_features'].float().contiguous()
        student_projected = outputs['student_projected'].float().contiguous()
        
        # 1. Knowledge Distillation Loss with stability
        nkd_loss_fn = NKDLoss(alpha=self.alpha, lambda_temp=1.0)
        kd_loss = nkd_loss_fn(student_logits, teacher_logits, labels)

        # 2. Improved Feature Alignment Loss
        feature_loss = self.feature_alignment_loss(student_projected, teacher_features)
        
        # 3. Cross Entropy Loss with label smoothing
        ce_loss = F.cross_entropy(student_logits, labels, label_smoothing=0.1)
        
        # 4. Improved Contrastive Loss
        cont_loss = self.contrastive_loss(student_projected, labels)
        
        # 5. Teacher Loss (for monitoring only)
        teacher_loss = F.cross_entropy(teacher_logits, labels)
        
        # Weighted sum of losses
        total_loss = (
            self.alpha * kd_loss +
            self.beta * feature_loss +
            self.gamma * ce_loss +
            self.contrastive_weight * cont_loss
        )
        
        return total_loss, {
            'kd_loss': kd_loss.item(),
            'feature_loss': feature_loss.item(),
            'ce_loss': ce_loss.item(),
            'cont_loss': cont_loss.item(),
            'teacher_loss': teacher_loss.item(),
            'total_loss': total_loss.item()
        }

    @staticmethod
    def update_temperature(self, epoch, max_epochs):
        """
        Temperature annealing
        """
        self.temperature = max(0.07, 1.0 - (epoch / max_epochs) * 0.93)

In [18]:
def train_epoch(model, train_loader, criterion, optimizer, config, scheduler=None):

    model.train()

    total_loss = 0

    metrics = defaultdict(float)  # Tracks additional metrics

    scaler = GradScaler() if config.use_amp else None

    

    pbar = tqdm(train_loader, desc='Training', leave =True)

    

    for batch_idx, (images, labels) in enumerate(pbar):

        images, labels = images.to(config.device), labels.to(config.device)

        

        # Mixed precision training

        if config.use_amp:

            with autocast():

                outputs = model(images, return_features=True)

                loss, batch_metrics = criterion(outputs, labels)
                print(batch_metrics)

            scaler.scale(loss).backward()

            

            if config.max_grad_norm:

                scaler.unscale_(optimizer)

                torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)

            

            scaler.step(optimizer)

            scaler.update()

        else:

            outputs = model(images, return_features=True)
            loss, batch_metrics = criterion(outputs, labels)

            loss.backward()

            

            if config.max_grad_norm:

                torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)

            

            optimizer.step()

        

        optimizer.zero_grad()

        if scheduler:

            scheduler.step()

        

        # Update loss and metrics

        total_loss += loss.item()

        for k, v in batch_metrics.items():

            metrics[k] += v

        

        # Update progress bar with detailed information

        pbar.set_postfix({

            'loss': f'{loss.item():.4f}',

            'avg_loss': f'{total_loss / (batch_idx + 1):.4f}',

            'lr': scheduler.get_last_lr()[0] if scheduler else optimizer.param_groups[0]['lr']

        })

    

    # Normalize metrics over all batches

    metrics = {k: v / len(train_loader) for k, v in metrics.items()}

    return total_loss / len(train_loader), metrics

In [19]:
def compute_topk_accuracy(outputs, labels, k):

    """

    Computes the top-k accuracy for the given outputs and labels.

    

    Args:

        outputs (torch.Tensor or dict): Model outputs, expected as a tensor.

        labels (torch.Tensor): Ground truth labels.

        k (int): The 'k' in top-k accuracy.

    

    Returns:

        float: Top-k accuracy as a percentage.

    """


    outputs = outputs['student_logits']  # Update the key to match your dictionary structure

    

    _, top_k_preds = outputs.topk(k, dim=1)

    top_k_correct = top_k_preds.eq(labels.view(-1, 1).expand_as(top_k_preds))

    return top_k_correct.any(dim=1).float().mean().item()

In [20]:
def test_compute_topk_accuracy():
    # Create synthetic outputs (logits)
    batch_size = 8
    num_classes = 5
    outputs = {'student_logits': torch.randn(batch_size, num_classes)}

    # Create synthetic labels
    labels = torch.randint(0, num_classes, (batch_size,))

    # Test the function for k = 1 and k = 3
    for k in [1, 3]:
        top_k_accuracy = compute_topk_accuracy(outputs, labels, k)
        print(f"Top-{k} Accuracy: {top_k_accuracy * 100:.2f}%")

# Run the test
test_compute_topk_accuracy()

Top-1 Accuracy: 25.00%
Top-3 Accuracy: 37.50%


In [21]:
def validate_epoch(model, val_loader, criterion, config):

    model.eval()

    total_loss = 0

    metrics = defaultdict(float)

    

    pbar = tqdm(val_loader, desc='Validating')

    

    with torch.no_grad():  # Disable gradient computation

        for batch_idx, (images, labels) in enumerate(pbar):

            images, labels = images.to(config.device), labels.to(config.device)

            

            # Forward pass

            outputs = model(images, return_features=True)

            loss, batch_metrics = criterion(outputs, labels)


            # Accumulate loss and metrics

            total_loss += loss.item()

            metrics['top1_acc'] += compute_topk_accuracy(outputs, labels, 1)

            metrics['top5_acc'] += compute_topk_accuracy(outputs, labels, 5)

            metrics['top10_acc'] += compute_topk_accuracy(outputs, labels, 10)

            

            # Update progress bar

            pbar.set_postfix({

                'loss': f'{loss.item():.4f}',

                'avg_loss': f'{total_loss / (batch_idx + 1):.4f}'

            })

    

    # Normalize metrics over all batches

    metrics = {k: v / len(val_loader) for k, v in metrics.items()}

    return total_loss / len(val_loader), metrics

In [22]:
from collections import defaultdict

In [23]:
train_ds = GoogleLandmarksDataset(train_df, class_name_to_idx=class_name_to_idx, transform=transforms_train)
train_dl = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    pin_memory=True,
    drop_last=True,
    num_workers=N_WORKERS
)

In [24]:
teacher_model = GoogleLandmarksModel(text_embeddings=text_embeddings, model_name="RN101", device=device)
teacher_model = teacher_model.float()

checkpoint = torch.load('/kaggle/input/k/thangchu/training-clip-baseline-on-landmarks-index-set/clip_landmark_results/best_val_loss_model_fold-0.pth', weights_only = False)
teacher_model.load_state_dict(checkpoint['model_state_dict'])
teacher_model.to(device)

student_model = timm.create_model("resnet18", pretrained=True, num_classes=N_CLASSES)
student_model = student_model.float()
checkpoint = torch.load('/kaggle/input/checkpointss/pytorch/default/1/best_student_model_fold0.pth',weights_only = False)
student_model.load_state_dict(checkpoint['model_state_dict'])

# Initialize the improved distillation model

distillation_model = CrossModalDistillationModel(

    teacher_model=teacher_model,

    student_model=student_model,

    text_embeddings=text_embeddings.to(device),

    projection_dim=512  # Same dimension as CLIP's feature space

)

distillation_checkpoint_path = '/kaggle/input/checkpointss/pytorch/default/1/best_model_fold0.pth'
distillation_checkpoint = torch.load(distillation_checkpoint_path, weights_only = False ,map_location=device)

# Load the state_dict into the distillation model
distillation_model.load_state_dict(distillation_checkpoint['model_state_dict'])
distillation_model.to(device)

# (Optional) Load optimizer state if continuing training


# Optimizer with weight decay

optimizer = AdamW([

    {'params': distillation_model.student.parameters(), 'lr': 1e-4},

    {'params': distillation_model.projection_head.parameters(), 'lr': 1e-4}

], weight_decay=0.01)

optimizer.load_state_dict(distillation_checkpoint['optimizer_state_dict'])



# Scheduler with warmup

num_training_steps = len(train_dl) * EPOCHS

num_warmup_steps = min(1000, num_training_steps // 20)  # 10% of training steps or 1000, whichever is smaller

scheduler = get_cosine_schedule_with_warmup(

    optimizer, 

    num_warmup_steps=num_warmup_steps,

    num_training_steps=num_training_steps

)



# Training loop with validation

patience = EARLY_STOPPING_EPOCH

patience_counter = 0



scaler = GradScaler() if USE_AMP else None



sfk = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

training_folds = [0] 

fold_metrics = []

for fold, (trn_idx, val_idx) in enumerate(sfk.split(train_df, train_df.landmark_id.tolist())):
    if fold not in training_folds:
        continue

    print(f"{'#'*30}\nStarting Fold {fold}\n{'#'*30}")

    # Prepare datasets and dataloaders
    df_train = train_df.iloc[trn_idx]
    df_valid = train_df.iloc[val_idx]

    train_ds = GoogleLandmarksDataset(df_train, class_name_to_idx=class_name_to_idx, transform=transforms_train)
    valid_ds = GoogleLandmarksDataset(df_valid, class_name_to_idx=class_name_to_idx, transform=transforms_val)

    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=N_WORKERS, drop_last=True, pin_memory=True)
    valid_dl = DataLoader(valid_ds, batch_size=BATCH_SIZE*2, shuffle=False, num_workers=N_WORKERS, drop_last=False, pin_memory=True)

    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(EPOCHS):
        train_loss, train_metrics = train_epoch(
            model=distillation_model,
            train_loader=train_dl,
            criterion=EnhancedDistillationLoss(Config()),
            optimizer=optimizer,
            config=Config(),
            scheduler=scheduler
        )

        val_loss, val_metrics = validate_epoch(
            model=distillation_model,
            val_loader=valid_dl,
            criterion=EnhancedDistillationLoss(Config()),
            config=Config()
        )

        print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
              f"Top1 Acc: {val_metrics['top1_acc']:.2f} | Top5 Acc: {val_metrics['top5_acc']:.2f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save({
        'model_state_dict': distillation_model.student.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'val_loss': val_loss,
        'epoch': epoch,
    }, f'{OUTPUT_DIR}/best_student_model_fold{fold}.pth')
            torch.save({
                'fold': fold,
                'epoch': epoch,
                'model_state_dict': distillation_model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,
            }, f'{OUTPUT_DIR}/best_model_fold{fold}.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}")
                break
    
    fold_metrics.append({
        'fold': fold,
        'train_loss': train_loss,
        'val_loss': val_loss,
        'top1_acc': val_metrics['top1_acc'],
        'top5_acc': val_metrics['top5_acc'],
        'top10_acc': val_metrics['top10_acc'],
    })

# Print overall metrics
print("Summary of Fold Metrics:")
for metric in fold_metrics:
    print(metric)

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

##############################
Starting Fold 0
##############################


Training:   0%|          | 0/1875 [00:00<?, ?it/s]/tmp/ipykernel_23/959980901.py:61: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():
Validating: 100%|██████████| 235/235 [02:12<00:00,  1.78it/s, loss=7.7995, avg_loss=7.3338]


Epoch 1 | Train Loss: 8.2833 | Val Loss: 7.3338 | Top1 Acc: 0.17 | Top5 Acc: 0.24


Validating: 100%|██████████| 235/235 [01:50<00:00,  2.14it/s, loss=7.7649, avg_loss=7.3276]


...

Epoch 36 | Train Loss: 8.0346 | Val Loss: 7.2906 | Top1 Acc: 0.19 | Top5 Acc: 0.25


Validating: 100%|██████████| 235/235 [01:50<00:00,  2.13it/s, loss=7.8279, avg_loss=7.2911]


Epoch 37 | Train Loss: 8.0315 | Val Loss: 7.2911 | Top1 Acc: 0.19 | Top5 Acc: 0.25


Validating: 100%|██████████| 235/235 [01:50<00:00,  2.13it/s, loss=7.8243, avg_loss=7.2894]

Epoch 38 | Train Loss: 8.0244 | Val Loss: 7.2894 | Top1 Acc: 0.19 | Top5 Acc: 0.25
Early stopping triggered at epoch 38
Summary of Fold Metrics:
{'fold': 0, 'train_loss': 8.024432688649496, 'val_loss': 7.28944915203338, 'top1_acc': 0.18743085106636614, 'top5_acc': 0.25225797872594063, 'top10_acc': 0.2754295212791321}
